## Probability for completely unfair stats when rolling ability scores using 4d6 drop lowest for D&D

[StackExchange question.](https://stats.stackexchange.com/questions/110671/probability-for-completely-unfair-stats-when-rolling-ability-scores-using-4d6-dr)

> What's the probability that, given 5 players, one player will have a highest ability score equal or lower than the lowest ability score of another player?

### Step 1: 4d6 keep highest 3

The problem of keeping the highest can be computed through a variety of methods ([example](https://stats.stackexchange.com/questions/130025/formula-for-dropping-dice-non-brute-force/242839)), but even simply enumerating all $6^4 = 1296$ possibilities is trivial for a computer as well. However you prefer to do it, this should be the first step.

In [1]:
import piplite
await piplite.install("icepool")

import icepool

one_ability = icepool.d6.highest(4, 3)

### Step 2: Lowest and highest ability scores for a single player

With the distribution of 4d6kh3 in hand, we then proceed to find the joint distribution of the lowest and highest ability scores for a single player.
We can do this inductively: given the distribution after the first $n$ scores, we can take the joint distribution with one additional (independent) score
and use this to compute the distribution for $n+1$ scores.

In [2]:
def append_score(current, score):
    # As we see each score, we track the lowest and highest seen so far.
    lo, hi = current
    return min(lo, score), max(hi, score)

# Base case: lo = 18 and hi = 3 which will be overwritten in the first iteration.
initial = icepool.Die([(18, 3)])

# Iteratively compute the result.
num_scores = 6
one_player = initial
for i in range(num_scores):
    # apply() takes care of iterating over outcomes and tracking the weights.
    one_player = icepool.apply(append_score, one_player, one_ability)
    
# Equivalent one-liner. reduce() is analogous to the functools version.
one_player = icepool.reduce(append_score, [one_ability] * num_scores, initial=initial)

# Alternative: make a pool that keeps the lowest and highest, and use the expand method.
one_player = one_ability.pool(num_scores)[1, ..., 1].expand()

### Step 3: Highest low score and lowest high score

We can repeat a similar strategy to find the highest of all low scores and the lowest of all high scores.

In [3]:
def append_player(a, b):
    # This time, we track the highest of low scores and lowest of high scores.
    return max(a[0], b[0]), min(a[1], b[1])

# Base case: hi_of_lo = 3 and lo_of_hi = 18 which will be overwritten
# in the first iteration.
initial = icepool.Die([(3, 18)])

# Iteratively compute the result.
print("Chance that some player's scores are all < another player's scores.")
result = initial
for i in range(6):
    result = icepool.apply(append_player, result, one_player)
    if i > 0:
        print(i + 1, 'players:', result.map(lambda x: x[0] > x[1]))


Chance that some player's scores are all < another player's scores.
2 players: Die with denominator 22452257707354557240087211123792674816

| Outcome | Probability |
|:--------|------------:|
| False   |  99.890635% |
| True    |   0.109365% |


3 players: Die with denominator 106387358923716524807713475752456393740167855629859291136

| Outcome | Probability |
|:--------|------------:|
| False   |  99.684993% |
| True    |   0.315007% |


4 players: Die with denominator 504103876157462118901767181449118688686067677834070116931382690099920633856

| Outcome | Probability |
|:--------|------------:|
| False   |  99.393990% |
| True    |   0.606010% |


5 players: Die with denominator 2388636399360109977557402041718133080829429159844757507642063199359529632522467783435119230976

| Outcome | Probability |
|:--------|------------:|
| False   |  99.026919% |
| True    |   0.973081% |


6 players: Die with denominator 1131827013876368608372040761544094055288406116057269606318770339914230540841